In [1]:
from tqdm import tqdm_notebook
import numpy as np
import datetime as dt
from google.cloud import storage
import json

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
class FeaturedNote:
    
    def __init__(self, pitch, wait, nb_pitches):
        self.pitch = pitch
        self.wait = wait
        self.nb_pitches = nb_pitches
    
    @classmethod
    def create_with_pitch_clipping(cls, pitch, wait, min_pitch, max_pitch):
        nb_pitches = max_pitch - min_pitch + 1
        if pitch is None:
            # Starting note, use special out-of-bounds value
            pitch = max_pitch - min_pitch
        else:
            pitch = constrain_pitch(pitch, min_pitch, max_pitch) - min_pitch
            assert 0 <= pitch < max_pitch - min_pitch
        
        return cls(pitch, wait, nb_pitches)
        
    def __repr__(self):
        return 'FeaturedNote(pitch={pitch}, wait={wait})'.format(**self.__dict__)
    
    def to_tuple(self):
        return (self.pitch, self.wait, self.nb_pitches)
    
    @classmethod
    def from_tuple(cls, tpl):
        return cls(*tpl)
        
    def calculate_features(self):
        pitch_features = [0] * self.nb_pitches
        pitch_features[self.pitch] = 1
        self.features = np.array(pitch_features + [self.wait])
        self.pitch_label = np.array(pitch_features)

In [3]:
def load_featured_songs_from_gs(filename):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket('verbatim')
    blob = bucket.blob('midi/data/featured_songs/{}'.format(filename))
    s = blob.download_as_string().decode('utf-8')
    lines = s.split('\n')
    print('Downloaded {lines} lines with metadata: {md}'.format(lines=len(lines) - 2, md=lines[0]))
    fsongs = []
    for line in tqdm_notebook(lines[1:-1]):
        tpls = json.loads(line)
        fsongs.append([FeaturedNote.from_tuple(tpl) for tpl in tpls])
    
    return fsongs

In [4]:
validation_fsongs = load_featured_songs_from_gs('validation_v2_2019-06-30_14:15:02.txt')

Downloaded 137 lines with metadata: {"max_pitch": 85, "version": 2, "min_pitch": 45}


In [5]:
train_fsongs = load_featured_songs_from_gs('train_v2_2019-06-30_14:15:19.txt')

Downloaded 967 lines with metadata: {"max_pitch": 85, "version": 2, "min_pitch": 45}


In [6]:
train_fsongs[0][0:30]

[FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=40, wait=0),
 FeaturedNote(pitch=22, wait=1.0),
 FeaturedNote(pitch=27, wait=0.8007812499999999),
 FeaturedNote(pitch=16, wait=0.3632812500000001),
 FeaturedNote(pitch=22, wait=0.0),
 FeaturedNote(pitch=26, wait=0.0),
 FeaturedNote(pitch=33, wait=0.0),
 FeaturedNote(pitch=34, wait=1.9518229166666665),
 FeaturedNote(pitch=15, wait=0.645833333333333),
 FeaturedNote(pitch=22, wait=0.0),
 FeaturedNote(pitch=25, wait=0.0),
 FeaturedNote(pitch=34, wait=0.0),
 FeaturedNote(pitch=30, wait

In [7]:
def extract_sequences_for_pitch_label(featured_songs, nb_notes_history):
    
    nb_datapoints = sum(len(fsong) - nb_notes_history for fsong in featured_songs)
    
    sequences = -1 * np.ones(shape=(nb_datapoints, nb_notes_history, nb_features))
    pitch_labels = -1 * np.ones(shape=(nb_datapoints, nb_pitches))
    
    data_index = 0
    
    for fsong in tqdm_notebook(featured_songs):
        
        for fnote in fsong:
            fnote.calculate_features()
            
        for i in range(nb_notes_history, len(fsong)):
            sequences[data_index] = np.array([fnote.features for fnote in fsong[i - nb_notes_history:i]])
            pitch_labels[data_index] = fsong[i].pitch_label
            data_index += 1
    
    return sequences, pitch_labels

In [8]:
# Note featuring parameters
min_pitch = 45  # inclusive
max_pitch = 85  # exclusive
nb_pitches = max_pitch - min_pitch + 1  # including out of bounds pitch
nb_features = nb_pitches + 1  # including scalar continuous wait feature

# Sequence featuring parameters
nb_notes_history = 16

In [9]:
validation_sequences, validation_pitch = extract_sequences_for_pitch_label(validation_fsongs, nb_notes_history)

In [10]:
train_sequences, train_pitch = extract_sequences_for_pitch_label(train_fsongs, nb_notes_history)

In [11]:
np.min(train_sequences)

0.0

In [12]:
np.min(train_pitch)

0.0

In [13]:
train_sequences.shape

(5726876, 16, 42)

In [14]:
train_pitch.shape

(5726876, 41)

In [18]:
model = Sequential()

# Recurrent layer
model.add(LSTM(32, input_shape=(nb_notes_history, nb_features),
               return_sequences=False, dropout=0.1, recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(32, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(nb_pitches, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
run_time = dt.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')

In [20]:
# Create callbacks
callbacks = [keras.callbacks.TensorBoard(log_dir='tb_logs/pitch_model_{}'.format(run_time), histogram_freq=0, write_graph=True, write_grads=False, 
                                         write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, 
                                         embeddings_data=None, update_freq='batch'),
             EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint('models/pitch_model_{}.h5'.format(run_time), save_best_only=True, save_weights_only=False)]

In [ ]:
history = model.fit(train_sequences,  train_pitch, 
                    batch_size=4096, epochs=50,
                    callbacks=callbacks,
                    validation_data=(validation_sequences, validation_pitch))

Train on 5726876 samples, validate on 639425 samples
Epoch 1/50
 913408/5726876 [===>..........................] - ETA: 6:53 - loss: 3.3790 - accuracy: 0.0888